In [ ]:
# gemma model[Zero Short Prompting]

In [ ]:
!pip install -q transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import Dataset
import pandas as pd

model_name = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # or "auto"
)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
df = pd.read_csv("dev.enhi.df.short.tsv", sep="\t")
df = df.rename(columns={"original": "source", "translation": "translation"})  # adjust if necessary


In [ ]:
def create_prompt(example):
    return f"""You are a machine translation evaluator.

Evaluate the quality of the translation from English to Hindi.

Source: {example['source']}
Translation: {example['translation']}

Rate the translation quality on a scale from 0 (completely incorrect) to 100 (perfect translation).
Output only the score as a number between 0 and 100."""

df["prompt"] = df.apply(create_prompt, axis=1)
dataset = Dataset.from_pandas(df)




In [ ]:
BATCH_SIZE = 4  # Adjust based on your GPU

def run_batch_inference(batch):
    inputs = tokenizer(batch["prompt"], return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    scores = []
    for output in decoded:
        last_line = output.strip().split("\n")[-1].strip()
        try:
            score = float(last_line)
        except ValueError:
            score = None
        scores.append(score)
    return {"predicted_score": scores}

dataset = dataset.map(run_batch_inference, batched=True, batch_size=BATCH_SIZE)


Parameter 'function'=<function run_batch_inference at 0x7c4454559080> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
final_df = dataset.to_pandas()
final_df.to_csv("gemma_predicted_scores.tsv", sep="\t", index=False)



In [ ]:
final_df.head()

,index,source,translation,scores,mean,z_scores,z_mean,prompt,predicted_score
0,0,In the flood-prone districts of the Netherland...,"नीदरलैंड के बाढ़ संभावित जिलों में, विशेष रूप ...","[90, 90, 79, 81]",85.0,"[0.10844457902530406, 0.17855383580414114, 0.6...",0.399822,You are a machine translation evaluator.\n\nEv...,None
1,1,Group A Group B The top five run scorers (tota...,ग्रुप ए ग्रुप बी शीर्ष पांच रन स्कोरर (कुल रन)...,"[95, 95, 87, 89]",91.5,"[0.5387802100780963, 0.6151157815355373, 1.520...",1.001232,You are a machine translation evaluator.\n\nEv...,None
2,2,"The final finished as a draw, with Essex winni...","मैच की पहली पारी में बढ़त हासिल करने के बाद, ए...","[95, 95, 70, 60]",80.0,"[0.5387802100780963, 0.6151157815355373, -0.22...",-0.085456,You are a machine translation evaluator.\n\nEv...,None
3,3,These traits—establishment of a working method...,ये विशेषताएं-कला का अभिन्न अंग कार्य प्रणाली क...,"[90, 90, 78, 78]",84.0,"[0.10844457902530406, 0.17855383580414114, 0.5...",0.306865,You are a machine translation evaluator.\n\nEv...,None
4,4,"Its two most important members, Britain and Fr...","इसके दो सबसे महत्वपूर्ण सदस्य, ब्रिटेन और फ्रा...","[90, 90, 88, 86]",88.5,"[0.10844457902530406, 0.17855383580414114, 1.6...",0.742850,You are a machine translation evaluator.\n\nEv...,None


In [ ]:
final_df['predicted_score'].value_counts()

,count
predicted_score,
